<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=978e396aceaed88a6ed505ca1e8653f3a67fbca33bcfa3f1627b4e2ab541a542
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1
    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-12-02 12:23:04
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.42 C
-------------------
Today PnL: -50.12 K (-0.35%)
Current PnL: -23.52 L (-15.41%)
CY Booked + Current PnL: -9.63 L (-6.31%)
-------------------
Total profit:  1.46 L
Total loss:  -24.98 L
-------------------
Total Booked + Current PnL: 17.35 L (13.81%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.76%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 88.93 L (62.51%)
Deployed:  1.26 C
Current:  1.42 C
CAGR/XIRR %: 7.0%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-0.45,-9.04,9.96,0.02,15815.0,-15777.0,158782.0,142.19,66.0,M-SC,13.05,234.0,-1.00,1.12,38.11,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.03,-16.79,20.29,0.09,17014.0,-16924.0,83853.0,94.66,36.0,M-SC,3.13,253.0,-0.99,0.59,10.16,OX40N,NTT,DURABLES
51,MASFIN,398.61,0.84,-2.94,25.74,22.05,24479.0,-2880.0,95100.0,-16.44,57.0,H-SC,6.60,164.0,-0.12,0.67,38.23,XR,ATH,FINANCE
43,ITC,452.00,-0.61,-1.83,12.49,10.44,24540.0,-3658.0,196480.0,-41.15,42.0,X-LC,1.40,5.0,-0.15,1.38,3.63,X40,NTT,FMCG
35,ICICIGI,2252.93,-0.69,5.51,13.92,20.19,25602.0,9606.0,183926.0,-17.20,46.0,X-MC,6.94,68.0,0.38,1.29,21.46,X40,ATH,INSURANCE


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
24,EASEMYTRIP,26.4,18.09,-53.53,213.54,45.70,175449.0,-94653.0,82162.0,-14.66,69.0,M-SC,23.62,205.0,-0.54,0.58,18.42,XY24,NTT,TRAVEL
13,BSOFT,831.7,5.03,-26.60,105.43,50.79,108843.0,-37416.0,103237.0,-5.68,65.0,H-SC,6.39,151.0,-0.34,0.73,20.20,XR,ATH,IT
0,5PAISA,593.0,3.37,-35.13,74.05,12.90,104952.0,-76765.0,141731.0,109.13,56.0,H-SC,12.24,161.0,-0.73,1.00,17.85,OX40N,NTT,FINANCE
84,WHIRLPOOL,2270.0,1.61,-19.36,125.60,81.92,116268.0,-22228.0,92570.0,-52.71,23.0,M-SC,3.94,236.0,-0.19,0.65,9.59,XR,NTT,DURABLES
57,RAJESHEXPO,518.0,1.53,-63.20,171.92,0.07,86789.0,-86695.0,50482.0,1749.75,58.0,L-SC,1.68,268.0,-1.00,0.35,25.55,OX40N,NTT,JEWELLERY


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-5.97,-19.64,84.73,48.45,136525.0,-39376.0,161130.0,-25.46,14.0,X-MC,8.26,64.0,-0.29,1.13,0.00,X40N,ATH,FINANCE
81,VALIANTORG,838.00,-2.76,-46.89,227.92,74.15,198033.0,-76718.0,86887.0,-256.58,33.0,H-SC,18.90,149.0,-0.39,0.61,11.38,XR,NTT,CHEMICALS
44,JCHAC,2282.00,-2.17,-30.59,44.06,-0.01,39086.0,-39095.0,88710.0,18052.17,12.0,M-SC,0.73,233.0,-1.00,0.62,0.45,OX40N,NTT,AC
38,INDIAMART,4810.62,-1.91,-1.76,110.42,106.72,133795.0,-2167.0,121169.0,-51.39,35.0,H-SC,1.98,139.0,-0.02,0.85,21.21,AR,ATH,MISC
26,GILLETTE,11206.78,-1.91,-0.58,33.19,32.42,83778.0,-1476.0,252420.0,-18.19,41.0,X-SC,8.03,85.0,-0.02,1.77,14.43,X40,ATH,FMCG


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
37,IEX,219.00,0.75,0.31,48.17,48.63,96612.0,611.0,200565.0,-33.16,64.0,H-SC,15.10,136.0,0.01,1.41,11.89,XR,NTT,MISC
86,ZYDUSLIFE,1286.17,0.58,-0.67,36.55,35.63,76082.0,-1414.0,208160.0,-14.53,45.0,H-MC,3.66,119.0,-0.02,1.46,15.88,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,-0.32,-5.26,119.46,107.91,167096.0,-7768.0,139876.0,-22.98,35.0,M-SC,10.97,216.0,-0.05,0.98,0.02,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,-0.45,-9.04,9.96,0.02,15815.0,-15777.0,158782.0,142.19,66.0,M-SC,13.05,234.0,-1.00,1.12,38.11,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.03,-16.79,20.29,0.09,17014.0,-16924.0,83853.0,94.66,36.0,M-SC,3.13,253.0,-0.99,0.59,10.16,OX40N,NTT,DURABLES
47,KANSAINER,340.00,-1.41,-23.63,48.58,13.47,100053.0,-63711.0,205956.0,-68.70,30.0,H-SC,1.99,158.0,-0.64,1.45,4.72,XY24,NTT,PAINTS
67,SIS,528.00,0.45,-25.53,62.84,21.26,51959.0,-28348.0,82684.0,1948.70,42.0,H-SC,3.89,166.0,-0.55,0.58,11.71,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.0,-0.07,2.58,67.93,72.27,118076.0,4372.0,173820.0,-11.98,60.0,M-LC,3.54,99.0,0.04,1.22,8.91,XR,NTT,IT
37,IEX,219.0,0.75,0.31,48.17,48.63,96612.0,611.0,200565.0,-33.16,64.0,H-SC,15.10,136.0,0.01,1.41,11.89,XR,NTT,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,-0.07,2.58,67.93,72.27,118076.0,4372.0,173820.0,-11.98,60.0,M-LC,3.54,99.0,0.04,1.22,8.91,XR,NTT,IT
37,IEX,219.00,0.75,0.31,48.17,48.63,96612.0,611.0,200565.0,-33.16,64.0,H-SC,15.10,136.0,0.01,1.41,11.89,XR,NTT,MISC
86,ZYDUSLIFE,1286.17,0.58,-0.67,36.55,35.63,76082.0,-1414.0,208160.0,-14.53,45.0,H-MC,3.66,119.0,-0.02,1.46,15.88,AR,ATH,PHARMA
38,INDIAMART,4810.62,-1.91,-1.76,110.42,106.72,133795.0,-2167.0,121169.0,-51.39,35.0,H-SC,1.98,139.0,-0.02,0.85,21.21,AR,ATH,MISC
25,FINCABLES,1641.55,-0.32,-5.26,119.46,107.91,167096.0,-7768.0,139876.0,-22.98,35.0,M-SC,10.97,216.0,-0.05,0.98,0.02,OX40N,ATH,CABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
9,BAJAJHFL,181.50,-5.97,-19.64,84.73,48.45,136525.0,-39376.0,161130.0,-25.46,14.0,X-MC,8.26,64.0,-0.29,1.13,0.00,X40N,ATH,FINANCE
11,BATAINDIA,2096.00,-1.17,-37.91,117.74,35.20,94070.0,-48774.0,79896.0,4.69,17.0,X-SC,13.97,92.0,-0.52,0.56,0.00,X40,NTT,FOOTWEAR
54,PAGEIND,51605.08,-0.41,-9.00,38.54,26.07,57425.0,-14740.0,149000.0,-33.02,27.0,X-MC,10.27,55.0,-0.26,1.05,0.00,X40,ATH,APPARELS
8,AWL,485.00,-1.76,-20.63,93.38,53.50,223658.0,-62237.0,239514.0,-61.59,30.0,X-MC,2.56,58.0,-0.28,1.68,6.38,XY24,NTT,FMCG
23,DMART,5391.45,-0.62,-6.40,36.88,28.12,72631.0,-13467.0,196940.0,-19.26,31.0,X-LC,4.41,19.0,-0.19,1.38,15.71,X40N,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,-0.61,-1.83,12.49,10.44,24540.0,-3658.0,196480.0,-41.15,42.0,X-LC,1.40,5.0,-0.15,1.38,3.63,X40,NTT,FMCG
3,ACC,3906.0,0.09,-22.16,110.88,64.15,205372.0,-52731.0,185220.0,-54.57,49.0,X-SC,1.52,82.0,-0.26,1.30,3.90,XY24,BTT,CEMENT
15,CAMS,4762.0,-1.90,0.46,24.21,24.79,61257.0,1168.0,253024.0,-4.14,45.0,X-SC,1.88,86.0,0.02,1.78,23.71,X40N,NTT,MISC
36,ICICIPRULI,790.0,-0.61,2.41,27.88,30.97,50463.0,4263.0,181001.0,-21.72,52.0,X-MC,2.16,75.0,0.08,1.27,15.28,X40,ATH,INSURANCE
66,SIEMENS,4671.5,0.81,-10.34,39.99,25.51,66723.0,-19245.0,166850.0,2.41,67.0,H-LC,2.36,49.0,-0.29,1.17,22.96,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-1.17,-37.91,117.74,35.20,94070.0,-48774.0,79896.0,4.69,17.0,X-SC,13.97,92.0,-0.52,0.56,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,-5.97,-19.64,84.73,48.45,136525.0,-39376.0,161130.0,-25.46,14.0,X-MC,8.26,64.0,-0.29,1.13,0.00,X40N,ATH,FINANCE
54,PAGEIND,51605.08,-0.41,-9.00,38.54,26.07,57425.0,-14740.0,149000.0,-33.02,27.0,X-MC,10.27,55.0,-0.26,1.05,0.00,X40,ATH,APPARELS
56,QUESS,424.00,-1.79,-29.47,102.53,42.84,47008.0,-19158.0,45848.0,-53.72,38.0,X-SC,37.46,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
55,PGHH,17907.65,-0.35,-4.83,40.54,33.76,77484.0,-9690.0,191130.0,-32.91,33.0,X-MC,4.26,57.0,-0.13,1.34,0.12,X40,ATH,FMCG


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-1.79,-29.47,102.53,42.84,47008.0,-19158.0,45848.0,-53.72,38.0,X-SC,37.46,83.0,-0.41,0.32,0.00,XY24,NTT,MISC
60,RELAXO,1176.00,-0.29,-47.37,189.37,52.31,144684.0,-68757.0,76403.0,-43.96,35.0,X-SC,4.91,91.0,-0.48,0.54,2.27,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-1.17,-37.91,117.74,35.20,94070.0,-48774.0,79896.0,4.69,17.0,X-SC,13.97,92.0,-0.52,0.56,0.00,X40,NTT,FOOTWEAR
52,MEDANTA,1486.00,-1.83,2.13,21.49,24.08,27862.0,2699.0,129649.0,-6.13,43.0,X-SC,5.90,89.0,0.10,0.91,22.16,XY24,NTT,HEALTHCARE
22,DIXON,18931.72,-0.05,-6.50,30.03,21.58,39349.0,-9108.0,131031.0,-52.88,35.0,X-MC,5.99,56.0,-0.23,0.92,16.00,X40N,ATH,IT


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.97,0.26,-14.64,39.74,19.28,117353.0,-50655.0,295301.0,-25.57,58.0,X-LC,6.91,1.0,-0.43,2.08,9.17,X40,ATH,IT
41,INFY,2275.00,0.35,6.84,44.95,54.87,150270.0,21416.0,334304.0,-16.73,62.0,X-LC,2.50,2.0,0.14,2.35,15.69,X40,BTT,IT
76,TMPV,600.00,-0.60,-14.31,65.93,42.18,156154.0,-39562.0,236848.0,-23.08,35.0,X-LC,2.75,3.0,-0.25,1.66,2.60,XY24,NTT,AUTO
82,VBL,671.64,-0.53,-2.72,39.46,35.67,121245.0,-8581.0,307261.0,-14.21,61.0,X-LC,5.45,4.0,-0.07,2.16,10.76,X40N,ATH,FMCG
43,ITC,452.00,-0.61,-1.83,12.49,10.44,24540.0,-3658.0,196480.0,-41.15,42.0,X-LC,1.40,5.0,-0.15,1.38,3.63,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
40,INDUSINDBK,1800.00,0.44,-37.18,111.54,32.89,54098.0,-28706.0,48501.0,-699.18,61.0,L-MC,5.92,259.0,-0.53,0.34,33.57,XR,NTT,BANKS
6,ASIANTILES,137.00,0.25,-15.06,113.90,81.67,90753.0,-14132.0,79678.0,7161.11,44.0,L-SC,19.03,271.0,-0.16,0.56,54.30,XR,NTT,CERAMICS
68,SONACOMS,806.63,-0.52,-12.45,59.38,39.54,52592.0,-12591.0,88568.0,-31.56,62.0,M-SC,6.63,220.0,-0.24,0.62,24.89,AR,ATH,AUTO
51,MASFIN,398.61,0.84,-2.94,25.74,22.05,24479.0,-2880.0,95100.0,-16.44,57.0,H-SC,6.60,164.0,-0.12,0.67,38.23,XR,ATH,FINANCE
70,SURYODAY,216.00,-1.48,-20.13,54.04,23.03,77290.0,-36047.0,143024.0,55.70,47.0,H-SC,9.82,167.0,-0.47,1.01,41.91,XR,NTT,BANKS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,-0.45,-9.04,9.96,0.02,15815.0,-15777.0,158782.0,142.19,66.0,M-SC,13.05,234.0,-1.00,1.12,38.11,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,1.51,1.74,18.87,20.95,48337.0,4391.0,256159.0,0.77,76.0,X-LC,16.47,31.0,0.09,1.80,38.00,X40,ATH,PAINTS
13,BSOFT,831.70,5.03,-26.60,105.43,50.79,108843.0,-37416.0,103237.0,-5.68,65.0,H-SC,6.39,151.0,-0.34,0.73,20.20,XR,ATH,IT
0,5PAISA,593.00,3.37,-35.13,74.05,12.90,104952.0,-76765.0,141731.0,109.13,56.0,H-SC,12.24,161.0,-0.73,1.00,17.85,OX40N,NTT,FINANCE
50,LTIM,7201.88,-0.60,11.80,17.76,31.66,48875.0,29047.0,275198.0,7.45,66.0,H-LC,14.42,43.0,0.59,1.93,52.69,X200,ATH,IT


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.98
1,25,44.80
2,50,75.62


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.40
MC    28.99
LC    25.57
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.72
X40      21.22
X40N     11.94
XR        9.64
XY25      9.12
AR        9.08
OX40N     7.59
X200      1.93
SR        0.97
MH        0.75
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.91
X-MC    22.49
X-LC    20.27
M-SC    11.92
X-SC     8.18
H-MC     4.77
H-LC     3.10
L-SC     1.39
M-MC     1.39
M-LC     1.22
L-LC     0.98
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.77,-5.37,39.45
IT,13.94,-13.94,72.77
FINANCE,9.45,-15.31,64.03
MISC,7.39,-23.61,74.25
ELECTRICAL,6.02,-9.54,49.65
PAINTS,5.94,-9.40,25.65
INSURANCE,4.40,-0.95,36.53
PHARMA,3.89,-0.75,33.00
AUTO,3.38,-19.43,70.64


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3092054.0,21
XR,1290549.0,13
AR,1280696.0,10
X40,1016526.0,14
X40N,749368.0,9
OX40N,710968.0,10
XY25,344075.0,6
SR,281917.0,2
MH,78077.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3579901.0,25
M-SC,1361815.0,15
X-MC,1280120.0,16
X-LC,878122.0,11
X-SC,755083.0,8
H-MC,400180.0,3
L-SC,262981.0,3
M-LC,118076.0,1
H-LC,115598.0,2


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1232939.0      6
           AR         883948.0      5
M-SC       XY24       777050.0      6
H-SC       XR         776293.0      7
X-MC       X40        453059.0      7
X-LC       X40        385619.0      5
X-MC       XY24       384639.0      3
H-SC       OX40N      326727.0      4
M-SC       OX40N      297452.0      5
X-SC       X40N       296993.0      3
H-SC       SR         281917.0      2
X-SC       XY24       280242.0      3
X-MC       X40N       258499.0      4
X-LC       XY24       225606.0      2
H-MC       AR         208602.0      2
X-LC       X40N       193876.0      2
H-MC       XY24       191578.0      1
X-MC       XY25       183923.0      2
X-SC       X40        177848.0      2
L-SC       XR         176192.0      2
M-SC       XR         165890.0      2
           AR         121423.0      2
M-LC       XR         118076.0      1
L-SC       OX40N       86789.0      1
H-SC       MH          78077.0      1
X-LC       XY25        73021.0      2
H-LC       AR          66723.0      1
L-MC       XR          54098.0      1
H-LC       X200        48875.0      1
M-MC       XY25        47684.0      1
L-LC       XY25        39447.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
